In [1]:
!pip install fastcoref

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00

In [2]:
import pandas as pd
from fastcoref import LingMessCoref
from tqdm.auto import tqdm

tqdm.pandas()

In [22]:
model = LingMessCoref("biu-nlp/lingmess-coref", device="cuda", enable_progress_bar=True)

In [4]:
data = pd.read_csv("./drive/MyDrive/raw_data2.csv")

In [11]:
data

,text
0,The system of transliteration from Cyrillic us...
1,The history of Russia in the twentieth century...
2,great divide between Soviet East and capitalis...
3,workers and a cultural straitjacket on the int...
4,Gorbachev revolution this reading of Soviet hi...
...,...
978,Her slender neck is cut by the merciless noose...
979,"As has been noted, it was not accidental that ..."
980,"29 Katherine Hodgson, “Soviet Women’s Poetry o..."
981,"Women in the Soviet War Effort, 1941–1945 243"


In [25]:
from IPython.display import clear_output

def coref_text(text: str, result):
    clusters = result.get_clusters(as_strings=False)

    replacements = []
    for cluster in clusters:
        start0, end0 = cluster[0]
        antecedent = text[start0:end0]
        for start, end in cluster[1:]:
            replacements.append((start, end, antecedent))


    replacements.sort(key=lambda x: x[0], reverse=True)

    new_text = list(text)
    for start, end, rep in replacements:
        new_text[start:end] = rep

    return "".join(new_text)

In [23]:
results = model.predict(texts=data["text"].tolist())

Map:   0%|          | 0/983 [00:00<?, ? examples/s]

Inference:   0%|          | 0/983 [00:00<?, ?it/s]

In [24]:
texts_with_result = zip(data["text"].tolist(), results)

In [26]:
processed_results = [coref_text(text, result) for text, result in tqdm(texts_with_result)]

0it [00:00, ?it/s]

In [29]:
data.text = processed_results

In [31]:
data.to_csv("./drive/MyDrive/raw_coref_data.csv", index=False)